<a href="https://colab.research.google.com/github/Buchunwang/UK-CLI/blob/main/UK_CLI_Gibbs_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the Gibbs sampling process for UK CLI data after Kalman filter and Hamilton (1989) basic filter.

---
First, we import  

1.   $Δ c_t$ including $Δ c_{t-1}, Δ c_{t-2}\ $
2.   $Δ y_{it}$ for $i=1,2,3,4,5$  
3.   $S_t$

That's all the import for Gibbs sampling,

In [1]:
import os
import pandas as pd
import numpy as np
url = 'https://raw.githubusercontent.com/Buchunwang/UK-CLI/main/Gibbs%20Sampling/St.csv'
print('From URL:', url)
St = pd.read_csv(url, header=None, encoding='utf-8', skiprows=1)#St
url = 'https://raw.githubusercontent.com/Buchunwang/UK-CLI/main/Gibbs%20Sampling/gibbspre.csv'
print('From URL:', url)
inputkal1 = pd.read_csv(url, header=None, encoding='utf-8', skiprows=1)#delta_ct delta_ct-1 delta_ct-2 delta_y_{1,2,3,4,5}t

From URL: https://raw.githubusercontent.com/Buchunwang/UK-CLI/main/Gibbs%20Sampling/St.csv
From URL: https://raw.githubusercontent.com/Buchunwang/UK-CLI/main/Gibbs%20Sampling/gibbspre.csv


In [2]:
import numpy as np

iter = 10000 # number of iterations
burn = 2000 #we burn the first 2000 iterations and sampling from the later 8000 iterations in the Monter Carlo
N = 419  # time point, there are two lags for delta_ct, so after preparing data from 421 data points, there are 419 left

#Now we begin to set the initial values of parameters wwe are going to estimate in the algorithm
phi_1 = np.random.normal(0, 1, (N - 4, 1))
phi_2 = np.random.normal(0, 1, (N - 4, 1))
lamda_1 = np.random.normal(0, 1, (N, 1))
#these are just initial settings, 
#later on we will draw samples of thses parameters by newly generated mean and variance (or other distribution parameters for other types of distribution)
lamda_2 = np.random.normal(0, 1, (N, 1))
lamda_3 = np.random.normal(0, 1, (N, 1))
lamda_4 = np.random.normal(0, 1, (N, 1))
lamda_5 = np.random.normal(0, 1, (N, 1))
Psi1 = np.random.normal(0, 1, (N - 2, 2))
Psi2 = np.random.normal(0, 1, (N - 2, 2))
Psi3 = np.random.normal(0, 1, (N - 2, 2))
Psi4 = np.random.normal(0, 1, (N - 2, 2))
Psi5 = np.random.normal(0, 1, (N - 2, 2))
rrighta3 = np.zeros((1, N - 2))#this matrix is for calculating distribution of lamda
q = np.random.binomial(0.9, 0.066)
p = np.random.binomial(0.967, 0.032)
sig1 = 0.2 #initial value of sigma_i
sig2 = 0.2
sig3 = 0.2
sig4 = 0.2
sig5 = 0.2
mu0 = np.random.normal(0, 1, (N - 2, 1))
mu1 = np.random.normal(0, 1, (N - 2, 1))
mut = mu0 + mu1 * St[2:N] #corresponding regime for t=3 to N

# variance and mean initial set
varlamda1 = 1
varlamda2 = 1
varlamda3 = 1
varlamda4 = 1
varlamda5 = 1
meanlamda1 = 0 #lamda_1~N(meanlamda1,varlamda1)
meanlamda2 = 0
meanlamda3 = 0
meanlamda4 = 0
meanlamda5 = 0
varpsi1 = np.eye(2)
varpsi2 = np.eye(2)
varpsi3 = np.eye(2)
varpsi4 = np.eye(2)
varpsi5 = np.eye(2)
meanpsi1 = np.zeros((2, 1))#psi_1~N(meanpsi1,varpsi1), meanpsi1=np.array([[a],[b]]),varpsii1=np.array([[a1,a2],[b1,b2]])
meanpsi2 = np.zeros((2, 1))
meanpsi3 = np.zeros((2, 1))
meanpsi4 = np.zeros((2, 1))
meanpsi5 = np.zeros((2, 1))
sig1a = 4 + N / 2 #sigma_1~IG(sig1a,sig1b) inverse gamma distribution
sig2a = 4 + N / 2
sig3a = 4 + N / 2
sig4a = 4 + N / 2
sig5a = 4 + N / 2
sig1b = 4
sig2b = 4
sig3b = 4
sig4b = 4
sig5b = 4
X1 = np.zeros((N - 2, 2))
X2 = np.zeros((N - 2, 2))
X3 = np.zeros((N - 2, 2))
X4 = np.zeros((N - 2, 2))
X5 = np.zeros((N - 2, 2))
varphi = np.array([[1, 0], [0, 1]])
meanphi = np.array([0, 0])
varmu = np.array([[1, 0], [0, 1]])
meanmu = np.array([[0], [0]])
Q = np.zeros((N - 4, 2))
Qstar = np.zeros((N - 4, 2))

# Output
# We set some empty matrices here to save values from each iteration 
# After the 2000 burn-in period, we need to save value for later 80000 iterations 
mlamda1 = np.zeros(8000)
mlamda2 = np.zeros(8000)
mlamda3 = np.zeros(8000)
mlamda4 = np.zeros(8000)
mlamda5 = np.zeros(8000)
vlamda1 = np.zeros(8000)
vlamda2 = np.zeros(8000)
vlamda3 = np.zeros(8000)
vlamda4 = np.zeros(8000)
vlamda5 = np.zeros(8000)
bsig1 = np.zeros(8000)
bsig2 = np.zeros(8000)
bsig3 = np.zeros(8000)
bsig4 = np.zeros(8000)
bsig5 = np.zeros(8000)
mpsi11 = np.zeros(8000)
mpsi21 = np.zeros(8000)
mpsi31 = np.zeros(8000)
mpsi41 = np.zeros(8000)
mpsi12 = np.zeros(8000)
mpsi22 = np.zeros(8000)
mpsi32 = np.zeros(8000)
mpsi42 = np.zeros(8000)
mpsi51 = np.zeros(8000)
mpsi52 = np.zeros(8000)
vpsi11 = np.zeros(8000)
vpsi21 = np.zeros(8000)
vpsi31 = np.zeros(8000)
vpsi41 = np.zeros(8000)
vpsi12 = np.zeros(8000)
vpsi22 = np.zeros(8000)
vpsi32 = np.zeros(8000)
vpsi42 = np.zeros(8000)
vpsi51 = np.zeros(8000)
vpsi52 = np.zeros(8000)
mphi1 = np.zeros(8000)
mphi2 = np.zeros(8000)
vphi1 = np.zeros(8000)
vphi2 = np.zeros(8000)
mmu1 = np.zeros(8000)
mmu2 = np.zeros(8000)
vmu1 = np.zeros(8000)
vmu2 = np.zeros(8000)


As the Gibbs sampling is a whole loop, we can't seperate it into blocks. So I will show the corresponding algorithms here.

---



---
  
    


1.  Generating $\lambda _i$, $i=1,2,3,4,5$
Given $\tilde{S}_T=\begin{bmatrix}
	S_1& S_2 &\cdots&S_T
\end{bmatrix}'$ from last step and $\tilde{c}_T=\begin{bmatrix}
	c_1& c_2 &\cdots&c_T
\end{bmatrix}'$.  

\begin{equation}
	y^*_{it}=\lambda_i\Delta c_t^*+\epsilon _{it}
\end{equation}
where $\Delta c^*_t=\Psi_i(L)\Delta c_t$ and $\Delta y^*_t=\Psi_i(L)\Delta y_t$. Define $\Delta \tilde{C}^*$ and $\Delta \tilde{y}^*_i$ as variable vectors of right-hand-side and left-hand-side individually. We can then generate posterior distribution $\lambda_i$ by:    
  

$$
\lambda_i \sim N\left(\left(\sigma_i^{-2} \Delta\tilde{C}^{\prime} \Delta\tilde{C}^{\ast} + X_i^{-1}\right)^{-1}\left(\sigma_i^{-2} \Delta\tilde{C}^{\prime} \Delta\tilde{y}_i^{\ast} + X_i^{-1} x_i\right), \left(\sigma_i^{-2} \Delta\tilde{C}^{\prime} \Delta\tilde{C}^{\ast} + X_i^{-1}\right)^{-1}\right)
$$

given prior distribution as 
\begin{equation}
	\lambda_i\sim N(x_i,X_i)
\end{equation}


---


2. Generating $\psi$ and $\sigma_i$, $i=1,2,3,4,5$
Let $Z_t=y_{it}-\lambda _i \Delta c_t=e_{it}$ where $\lambda _i$ is newly generated from last step.  

\begin{equation}
	\Psi(L)Z_t=\epsilon_{it}
\end{equation} 

\begin{equation}
	Z_t=\psi_{i1}Z_{t-1}+\psi{i2}Z_{t-2}+\epsilon_{it}
\end{equation} 

Define $\Delta \tilde{Z}$ and $\tilde{X}$ as variable vectors of right-hand-side and left-hand-side individually. We can then generate posterior distribution $\tilde\Psi_i$ ($\tilde\Psi_i=\begin{bmatrix}
	\psi_{i1}&\psi_{i2}
\end{bmatrix}'$) by:
$$
\tilde{\Psi}_i \sim N\left(\left(\sigma_i^{-2} \tilde{X}^{\prime} \tilde{X}^{\ast} + \Pi_i^{-1}\right)^{-1}\left(\sigma_i^{-2} \tilde{X}^{\prime} \tilde{Z}_i^{\ast} + \Pi_i^{-1} \pi_i\right),\left(\sigma_i^{-2} \tilde{X}^{\prime} \tilde{X}^{\ast} + \Pi_i^{-1}\right)^{-1}\right)
$$

given prior distribution as 
\begin{equation}
	\lambda_i\sim N(\pi _i\Pi _i)
\end{equation}
With new $\tilde\Psi_i$ from above,  

\begin{equation}
	\sigma^2_i\sim IG(\frac{a+T}{2},\frac{b}{2}+\frac{1}{2}(\tilde A-\tilde Z\tilde \Psi_i)'(\tilde A-\tilde Z\tilde \Psi_i))
\end{equation}
given prior $\sigma^2 \sim IG(\frac{a}{2},\frac{b}{2})$ where $IG$ means inverse gamma distribution. $a$ and $b$ are known value and we follow \citet{kim2} to set $a=b=4$.

---


3. Generating $\mu_0$, $\mu_1$ and $\tilde \phi$ 
 To generate
$\tilde\phi=\begin{bmatrix}\phi_1&\phi_2\end{bmatrix}'$, let $G_t=\Delta c_t-\mu_{s_t}$.   

\begin{equation}
	G_t=\phi_1G_{t-1}-\phi_2G_{t-2}+v_t
\end{equation}
let $\tilde Q$ and $\tilde G$ be right-hand-side and left-hand-side variable vector individually.
We can generate posterior distribution of $\Phi$. 

\begin{equation}
	\tilde \Phi\sim N((\tilde Q'\tilde Q+X^{-1})^{-1}(\tilde Q'\tilde G+X^{-1}x),(\tilde Q'\tilde Q+X^{-1})^{-1})
\end{equation}
given prior as 

\begin{equation}
	\Phi\sim N(x,X)
\end{equation}
With new $\Phi$ above, we let $G^*=\Delta c_t-\phi_1\Delta c_{t-1}-\phi_2\Delta c_{t-2}$, then we can get 


\begin{equation}
	G^*=\mu_0^*+\mu_1(S_t-\phi_1S_{t-1}-\phi_2S_{t-2})+v_t
\end{equation}  

where $\mu_0^*=\mu_0(1-\phi_1-\phi_2)$. Let $\tilde Q^*$ and $\tilde G^*$ be right-hand-side and left-hand-side variable vector individually. Posterior distribution of $\tilde \mu=\begin{bmatrix}
	\mu_0^*&\mu_t
\end{bmatrix}$ from. 

$$
\tilde{\mu} \sim N\left(\left(\tilde{Q}^{\ast\prime}\tilde{Q}^{\ast} + X^{*^{-1}}\right)^{-1}\left(\tilde{Q}^{\ast\prime}\tilde{G}^{\ast} + X^{*^{-1}}x^{\ast}\right), \left(\tilde{Q}^{\ast\prime}\tilde{Q}^{\ast} + X^{*^{-1}}\right)^{-1}\right)
$$

given prior as
\begin{equation}
	\Phi\sim N(x^*,X^*)
\end{equation}

The following iteration takes around 3 minutes in google colab

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import norm, gamma

for i in range(iter):
    righta3 = inputkal1.iloc[:, 0].T
    N = inputkal1.shape[0]
# Section 1
# form psi(L)*delta_y
    ys1 = (inputkal1.iloc[2:N, 3].values - inputkal1.iloc[1:N-1, 3].values * Psi1[:, 0] - inputkal1.iloc[0:N-2, 3].values * Psi1[:, 1]).reshape(-1, 1)
    ys2 = (inputkal1.iloc[2:N, 4].values - inputkal1.iloc[1:N-1, 4].values * Psi2[:, 0] - inputkal1.iloc[0:N-2, 4].values * Psi1[:, 1]).reshape(-1, 1)
    ys3 = (inputkal1.iloc[2:N, 5].values - inputkal1.iloc[1:N-1, 5].values * Psi3[:, 0] - inputkal1.iloc[0:N-2, 5].values * Psi1[:, 1]).reshape(-1, 1)
    ys4 = (inputkal1.iloc[2:N, 6].values - inputkal1.iloc[1:N-1, 6].values * Psi4[:, 0] - inputkal1.iloc[0:N-2, 6].values * Psi4[:, 1]).reshape(-1, 1)
    ys5 = (inputkal1.iloc[2:N, 7].values - inputkal1.iloc[1:N-1, 7].values * Psi5[:, 0] - inputkal1.iloc[0:N-2, 7].values * Psi5[:, 1]).reshape(-1, 1)

# delta C*~
    rrighta3 = (righta3[2:N].values - righta3[1:N-1].values * Psi1[:, 0] - righta3[0:N-2].values * Psi1[:, 1]).reshape(-1, 1)


# posterior distribution of lamda variance
    varlamda1 = 1 / (np.sum(rrighta3 * rrighta3.T) / (sig1 ** 2) + 1 / varlamda1)
    varlamda2 = 1 / (np.sum(rrighta3 * rrighta3.T) / (sig2 ** 2) + 1 / varlamda2)
    varlamda3 = 1 / (np.sum(rrighta3 * rrighta3.T) / (sig3 ** 2) + 1 / varlamda3)
    varlamda4 = 1 / (np.sum(rrighta3 * rrighta3.T) / (sig4 ** 2) + 1 / varlamda4)
    varlamda5 = 1 / (np.sum(rrighta3 * rrighta3.T) / (sig5 ** 2) + 1 / varlamda5)

# posterior distribution of lamda mean

    meanlamda1 = 1 / (np.sum(rrighta3 * rrighta3.T) / (sig1 ** 2) + 1 / varlamda1) * (meanlamda1 / varlamda1 + np.sum(rrighta3 * ys1.T) / (sig1 ** 2))
    meanlamda2 = 1 / (np.sum(rrighta3 * rrighta3.T) / (sig2 ** 2) + 1 / varlamda2) * (meanlamda2 / varlamda2 + np.sum(rrighta3 * ys2.T) / (sig2 ** 2))
    meanlamda3 = 1 / (np.sum(rrighta3 * rrighta3.T) / (sig3 ** 2) + 1 / varlamda3) * (meanlamda3 / varlamda3 + np.sum(rrighta3 * ys3.T) / (sig3 ** 2))
    meanlamda4 = 1 / (np.sum(rrighta3 * rrighta3.T) / (sig4 ** 2) + 1 / varlamda4) * (meanlamda4 / varlamda4 + np.sum(rrighta3 * ys4.T) / (sig4 ** 2))
    meanlamda5 = 1 / (np.sum(rrighta3 * rrighta3.T) / (sig5 ** 2) + 1 / varlamda5) * (meanlamda5 / varlamda5 + np.sum(rrighta3 * ys5.T) / (sig5 ** 2))

#generate new lamda using new mean & new variance
    lamda_1 = np.random.normal(meanlamda1, np.sqrt(varlamda1), (N, 1))
    lamda_2 = np.random.normal(meanlamda2, np.sqrt(varlamda2), (N, 1))
    lamda_3 = np.random.normal(meanlamda3, np.sqrt(varlamda3), (N, 1))
    lamda_4 = np.random.normal(meanlamda4, np.sqrt(varlamda4), (N, 1))
    lamda_5 = np.random.normal(meanlamda5, np.sqrt(varlamda5), (N, 1))

# Section 2
# generate new psi
# Form Z and X as shown in the algorithm overview
    Z1 = inputkal1.iloc[:, 3].values - inputkal1.iloc[:, 0].values * lamda_1.flatten()
    Z2 = inputkal1.iloc[:, 4].values - inputkal1.iloc[:, 0].values * lamda_2.flatten()
    Z3 = inputkal1.iloc[:, 5].values - inputkal1.iloc[:, 0].values * lamda_3.flatten()
    Z4 = inputkal1.iloc[:, 6].values - inputkal1.iloc[:, 0].values * lamda_3.flatten()
    Z5 = inputkal1.iloc[:, 7].values - inputkal1.iloc[:, 0].values * lamda_3.flatten()

    X1 = np.column_stack((Z1[1:N - 1], Z1[:N - 2]))
    X2 = np.column_stack((Z2[1:N - 1], Z2[:N - 2]))
    X3 = np.column_stack((Z3[1:N - 1], Z3[:N - 2]))
    X4 = np.column_stack((Z4[1:N - 1], Z4[:N - 2]))
    X5 = np.column_stack((Z5[1:N - 1], Z5[:N - 2]))

#save variance value of psi from last iteration for calculating mean of psi
#as variance would get upgraded before calculating mean

    fvarpsi1 = varpsi1
    fvarpsi2 = varpsi2
    fvarpsi3 = varpsi3
    fvarpsi4 = varpsi4
    fvarpsi5 = varpsi5

# form varpsi^-1 for the variance matrix from last iteration
# as a part of calculation for posterior distribution
    ffvarpsi1 = np.linalg.pinv(varpsi1)
    ffvarpsi2 = np.linalg.pinv(varpsi2)
    ffvarpsi3 = np.linalg.pinv(varpsi3)
    ffvarpsi4 = np.linalg.pinv(varpsi4)
    ffvarpsi5 = np.linalg.pinv(varpsi5)
# postrior variance of psi
    varpsi1 = np.linalg.pinv(np.linalg.pinv(varpsi1) + X1.T @ X1 * sig1)
    varpsi2 = np.linalg.pinv(np.linalg.pinv(varpsi2) + X2.T @ X2 * sig2)
    varpsi3 = np.linalg.pinv(np.linalg.pinv(varpsi3) + X3.T @ X3 * sig3)
    varpsi4 = np.linalg.pinv(np.linalg.pinv(varpsi4) + X4.T @ X4 * sig4)
    varpsi5 = np.linalg.pinv(np.linalg.pinv(varpsi5) + X5.T @ X5 * sig4)
# posterior mean of psi
    meanpsi1 = np.dot(np.linalg.pinv(np.linalg.pinv(varpsi1) + X1.T @ X1 / (sig1 ** 2)) , (np.linalg.pinv(np.linalg.pinv(varpsi1))@ meanpsi1 + (X1.T @ Z1[2:N]).reshape(-1, 1) / (sig1 ** 2)))
    meanpsi2 = np.dot(np.linalg.pinv(np.linalg.pinv(varpsi2) + X2.T @ X2 / (sig2 ** 2)) , (np.linalg.pinv(np.linalg.pinv(varpsi2))@ meanpsi2 + (X2.T @ Z2[2:N]).reshape(-1, 1) / (sig2 ** 2)))
    meanpsi3 = np.dot(np.linalg.pinv(np.linalg.pinv(varpsi3) + X3.T @ X3 / (sig3 ** 2)) , (np.linalg.pinv(np.linalg.pinv(varpsi3))@ meanpsi3 + (X3.T @ Z3[2:N]).reshape(-1, 1) / (sig3 ** 2)))
    meanpsi4 = np.dot(np.linalg.pinv(np.linalg.pinv(varpsi4) + X4.T @ X4 / (sig4 ** 2)) , (np.linalg.pinv(np.linalg.pinv(varpsi4))@ meanpsi4 + (X4.T @ Z4[2:N]).reshape(-1, 1) / (sig4 ** 2)))
    meanpsi5 = np.dot(np.linalg.pinv(np.linalg.pinv(varpsi5) + X5.T @ X5 / (sig5 ** 2)) , (np.linalg.pinv(np.linalg.pinv(varpsi5))@ meanpsi5 + (X5.T @ Z5[2:N]).reshape(-1, 1) / (sig5 ** 2)))
# sampling new psi with new mean & new variance for each psi_ij, i=1,2,3,4,5 j=1,2
    psi_11 = np.random.normal(meanpsi1[0, :], np.sqrt(varpsi1[0, 0]), size=(N-2, 1))
    psi_21 = np.random.normal(meanpsi2[0, :], np.sqrt(varpsi2[0, 0]), size=(N-2, 1))
    psi_31 = np.random.normal(meanpsi3[0, :], np.sqrt(varpsi3[0, 0]), size=(N-2, 1))
    psi_41 = np.random.normal(meanpsi4[0, :], np.sqrt(varpsi4[0, 0]), size=(N-2, 1))
    psi_51 = np.random.normal(meanpsi5[0, :], np.sqrt(varpsi5[0, 0]), size=(N-2, 1))

    psi_12 = np.random.normal(meanpsi1[1, :], np.sqrt(varpsi1[1, 1]), size=(N-2, 1))
    psi_22 = np.random.normal(meanpsi2[1, :], np.sqrt(varpsi2[1, 1]), size=(N-2, 1))
    psi_32 = np.random.normal(meanpsi3[1, :], np.sqrt(varpsi3[1, 1]), size=(N-2, 1))
    psi_42 = np.random.normal(meanpsi4[1, :], np.sqrt(varpsi4[1, 1]), size=(N-2, 1))
    psi_52 = np.random.normal(meanpsi5[1, :], np.sqrt(varpsi5[1, 1]), size=(N-2, 1))

# Form new psi_ij (with size (N,1)) to psi_i as a matrix of size (N,2), first column is psi_i1, second column is psi_i2
    Psi1 = np.column_stack((psi_21, psi_22))
    Psi2 = np.column_stack((psi_21, psi_22))
    Psi3 = np.column_stack((psi_31, psi_32))
    Psi4 = np.column_stack((psi_41, psi_42))
    Psi5 = np.column_stack((psi_51, psi_52))

#Generating sigma_i, i=1,2,3,4,5
    Xx1 = X1 * Psi1
    Xx2 = X2 * Psi2
    Xx3 = X3 * Psi3
    Xx4 = X4 * Psi4
    Xx5 = X5 * Psi5
# sigma~IG(sig1a,sig1b), posterior sig1a=4+N/2 and it's fiexed
# we only needs to calculate posterior sig1b
    sig1b = 4 + 0.5 * np.dot(((Z1[2:N] - Xx1[:, 0] - Xx1[:, 1]).reshape(-1, 1)).T, (Z1[2:N] - Xx1[:, 0] - Xx1[:, 1]).reshape(-1, 1))
    sig2b = 4 + 0.5 * np.dot(((Z2[2:N] - Xx2[:, 0] - Xx2[:, 1]).reshape(-1, 1)).T, (Z2[2:N] - Xx2[:, 0] - Xx2[:, 1]).reshape(-1, 1))
    sig3b = 4 + 0.5 * np.dot(((Z3[2:N] - Xx3[:, 0] - Xx3[:, 1]).reshape(-1, 1)).T, (Z3[2:N] - Xx3[:, 0] - Xx3[:, 1]).reshape(-1, 1))
    sig4b = 4 + 0.5 * np.dot(((Z4[2:N] - Xx4[:, 0] - Xx4[:, 1]).reshape(-1, 1)).T, (Z4[2:N] - Xx4[:, 0] - Xx4[:, 1]).reshape(-1, 1))
    sig5b = 4 + 0.5 * np.dot(((Z5[2:N] - Xx5[:, 0] - Xx5[:, 1]).reshape(-1, 1)).T, (Z5[2:N] - Xx5[:, 0] - Xx5[:, 1]).reshape(-1, 1))

# generate mew sigma_i with new sig_{i}b
    sig1 = np.sqrt(1 / np.random.gamma(sig1a, 1 / sig1b))
    sig2 = np.sqrt(1 / np.random.gamma(sig2a, 1 / sig2b))
    sig3 = np.sqrt(1 / np.random.gamma(sig3a, 1 / sig3b))
    sig4 = np.sqrt(1 / np.random.gamma(sig4a, 1 / sig4b))
    sig5 = np.sqrt(1 / np.random.gamma(sig5a, 1 / sig5b))

# Section 3
# generate phi

# Form G and Q as shown in the overview of section 3
    G = inputkal1.iloc[4:N, 0] - mut[2:N - 2].T
    Q[:, 0] = inputkal1.iloc[3:N-1, 0].values - np.reshape(mut[1:N-3], (-1, 1)).T
    Q[:, 1] = inputkal1.iloc[2:N-2, 1].values - np.reshape(mut[0:N-4], (-1, 1)).T
    Gg = G[0:N - 4]
# save variance from last iteration
    fvarphi = varphi
# calculate posterior variance of phi
    varphi = np.linalg.pinv(np.dot(Q.T, Q) + np.linalg.pinv(fvarphi))
    aA1 = np.array([meanphi[0] / fvarphi[0,0], meanphi[1] / fvarphi[1,1]]).reshape((2, 1))
    ffvarphi = np.array([fvarphi[0], fvarphi[1]])
    fff = np.linalg.pinv(varphi)
# there could be some 'inf' showing up at fff[0,1] and fff[1,0]
# so we need to reform the matrix
# the variance is still exactly same
# so there won't be any impact on calculating posterior distribution

    ffff = np.array([[fff[0, 0], 0], [0, fff[1, 1]]])

# calculating posterior mean of phi
    meanphi = np.linalg.inv(np.dot(Q.T, Q) + ffff) @ (np.dot(Q.T, Gg.T) + aA1)
# sampling new phi
    phi_1 = np.random.normal(meanphi[0], np.sqrt(varphi[0, 0]), size=(N - 2, 1))
    phi_2 = np.random.normal(meanphi[1], np.sqrt(varphi[1, 1]), size=(N - 2, 1))

# generating mu
# form G* and Q* as shown in the overview of section 3
    Gstar = inputkal1.iloc[4:N, 0].values.reshape(-1, 1) - phi_1[2:N - 2] * inputkal1.iloc[3:N - 1, 0].values.reshape(-1, 1) + phi_2[2:N - 2] * inputkal1.iloc[2:N - 2, 0].values.reshape(-1, 1)
    mu0s = mu0[2:N - 2] * (np.ones((N - 4, 1)) - phi_1[2:N - 2] - phi_2[2:N - 2])
    Qstar = np.zeros((N - 4, 2))
    Qstar[:, 0] = (mu0s + np.random.normal(0, 1, size=(N - 4, 1))).flatten()
    Qstar[:, 1] = (np.multiply(getattr(mut[2:N - 2], 'values', mut[2:N - 2]), getattr(St[2:N - 2], 'values', St[2:N - 2]))  - np.multiply(getattr(mut[2:N - 2], 'values', mut[2:N - 2]), getattr(phi_1[2:N - 2], 'values', phi_1[2:N - 2]) * getattr(St[1:N - 3], 'values', St[1:N - 3])) - np.multiply(getattr(mut[2:N - 2], 'values', mut[2:N - 2]), getattr(phi_2[2:N - 2], 'values', phi_2[2:N - 2]) * getattr(St[0:N - 4], 'values', St[0:N - 4])) + np.random.normal(0, 1, size=(N - 4, 1))).flatten()
#save variance of mu from last iteration
    fvarmu = varmu
# generate posterior variance of mu
    varmu = np.linalg.pinv(np.dot(Qstar.T, Qstar) + np.linalg.pinv(fvarmu))
    ffvarmu = np.linalg.pinv(fvarmu) # inverse of former variance matrix from last iteration
# same situation with phi,
# there could be some 'inf' showing up at ffvarmu[0,1] and ffvarmu[1,0]
# so we need to reform the matrix
# the variance is still exactly same
# there won't be any impact on calculating posterior distribution
    fffvarmu = np.array([[ffvarmu[0, 0], 0], [0, ffvarmu[1, 1]]])
# generate posterior mean value of mu
    meanmu = np.linalg.inv(fffvarmu + np.dot(Qstar.T, Qstar)) @ (meanmu / np.array([[fvarmu[0, 0], fvarmu[1, 1]]]).T + np.dot(Qstar.T, Gstar))



# same value of each iteration after first 2000 iterations
    if i>burn:
      mlamda1[i - burn] = meanlamda1
      mlamda2[i - burn] = meanlamda2
      mlamda3[i - burn] = meanlamda3
      mlamda4[i - burn] = meanlamda4
      mlamda5[i - burn] = meanlamda5
      vlamda1[i - burn] = varlamda1
      vlamda2[i - burn] = varlamda2
      vlamda3[i - burn] = varlamda3
      vlamda4[i - burn] = varlamda4
      vlamda5[i - burn] = varlamda5
      bsig1[i - burn] = sig1
      bsig2[i - burn] = sig2
      bsig3[i - burn] = sig3
      bsig4[i - burn] = sig4
      bsig5[i - burn] = sig5
      mpsi11[i - burn] = meanpsi1[0, 0]
      mpsi21[i - burn] = meanpsi2[0, 0]
      mpsi31[i - burn] = meanpsi3[0, 0]
      mpsi41[i - burn] = meanpsi4[0, 0]
      mpsi51[i - burn] = meanpsi5[0, 0]
      mpsi12[i - burn] = meanpsi1[1, 0]
      mpsi22[i - burn] = meanpsi2[1, 0]
      mpsi32[i - burn] = meanpsi3[1, 0]
      mpsi42[i - burn] = meanpsi4[1, 0]
      mpsi52[i - burn] = meanpsi5[1, 0]
      vpsi11[i - burn] = varpsi1[0, 0]
      vpsi21[i - burn] = varpsi2[0, 0]
      vpsi31[i - burn] = varpsi3[0, 0]
      vpsi41[i - burn] = varpsi4[0, 0]
      vpsi51[i - burn] = varpsi5[0, 0]
      vpsi12[i - burn] = varpsi1[1, 1]
      vpsi22[i - burn] = varpsi2[1, 1]
      vpsi32[i - burn] = varpsi3[1, 1]
      vpsi42[i - burn] = varpsi4[1, 1]
      vpsi52[i - burn] = varpsi5[1, 1]
      mphi1[i - burn] = meanphi[0, 0]
      mphi2[i - burn] = meanphi[1, 0]
      vphi1[i - burn] = varphi[0, 0]
      vphi2[i - burn] = varphi[1, 1]
      mmu1[i - burn] = meanmu[0, 0]
      mmu2[i - burn] = meanmu[1, 0]
      vmu1[i - burn] = varmu[0, 0]
      vmu2[i - burn] = varmu[1, 1]




In [4]:
import numpy as np
# after getting mean and variance from 8000 iterations, we take mean of 8000 iterations of mean value and variance value
mmlamda1 = np.mean(mlamda1)
mmlamda2 = np.mean(mlamda2)
mmlamda3 = np.mean(mlamda3)
mmlamda4 = np.mean(mlamda4)
mmlamda5 = np.mean(mlamda5)
vvlamda1 = np.mean(vlamda1)
vvlamda2 = np.mean(vlamda2)
vvlamda3 = np.mean(vlamda3)
vvlamda4 = np.mean(vlamda4)
vvlamda5 = np.mean(vlamda5)
mbsig1 = np.mean(bsig1)
mbsig2 = np.mean(bsig2)
mbsig3 = np.mean(bsig3)
mbsig4 = np.mean(bsig4)
mbsig5 = np.mean(bsig5)
mmpsi11 = np.mean(mpsi11)
mmpsi21 = np.mean(mpsi21)
mmpsi31 = np.mean(mpsi31)
mmpsi41 = np.mean(mpsi41)
mmpsi51 = np.mean(mpsi51)
mmpsi12 = np.mean(mpsi12)
mmpsi22 = np.mean(mpsi22)
mmpsi32 = np.mean(mpsi32)
mmpsi42 = np.mean(mpsi42)
mmpsi52 = np.mean(mpsi52)
vvpsi11 = np.mean(vpsi11)
vvpsi21 = np.mean(vpsi21)
vvpsi31 = np.mean(vpsi31)
vvpsi41 = np.mean(vpsi41)
vvpsi51 = np.mean(vpsi51)
vvpsi12 = np.mean(vpsi12)
vvpsi22 = np.mean(vpsi22)
vvpsi32 = np.mean(vpsi32)
vvpsi42 = np.mean(vpsi42)
vvpsi52 = np.mean(vpsi52)
mmphi1 = np.mean(mphi1)
mmphi2 = np.mean(mphi2)
vvphi1 = np.mean(vphi1)
vvphi2 = np.mean(vphi2)
mmmu1 = np.mean(mmu1)
mmmu2 = np.mean(mmu2)
vvmu1 = np.mean(vmu1)
vvmu2 = np.mean(vmu2)
sqrt_var_bsig1 = np.sqrt(np.var(bsig1))
sqrt_var_bsig2 = np.sqrt(np.var(bsig2))
sqrt_var_bsig3 = np.sqrt(np.var(bsig3))
sqrt_var_bsig4 = np.sqrt(np.var(bsig4))
sqrt_var_bsig5 = np.sqrt(np.var(bsig5))
#sigma is slightly special as it follows inverse Gamma distribution
#but Kim & Nelson (1998) still provides the mean and SD for sigma
# so what we do here is to calculate mean and variance of 80000 sigma_i for i=1,2,3,4,5 from 8000 iterations
mbsig1=np.mean(bsig1)
mbsig2=np.mean(bsig2)
mbsig3=np.mean(bsig3)
mbsig4=np.mean(bsig4)
mbsig5=np.mean(bsig5)

#print all results
print("mean of lamda1:", mmlamda1)
print("mean of lamda2:", mmlamda2)
print("mean of lamda3:", mmlamda3)
print("mean of lamda4:", mmlamda4)
print("mean of lamda5:", mmlamda5)
print("stanard deviation of lamda1:", np.sqrt(vvlamda1))
print("stanard deviation of lamda2:", np.sqrt(vvlamda2))
print("stanard deviation of lamda3:", np.sqrt(vvlamda3))
print("stanard deviation of lamda4:", np.sqrt(vvlamda4))
print("stanard deviation of lamda5:", np.sqrt(vvlamda5))
print("mean of psi11:", mmpsi11)
print("mean of psi21:", mmpsi21)
print("mean of psi31:", mmpsi31)
print("mean of psi41:", mmpsi41)
print("mean of psi51:", mmpsi51)
print("mean of psi12:", mmpsi12)
print("mean of psi22:", mmpsi22)
print("mean of psi32:", mmpsi32)
print("mean of psi42:", mmpsi42)
print("mean of psi52:", mmpsi52)
print("stanard deviation of psi11:", np.sqrt(vvpsi11))
print("stanard deviation of psi21:", np.sqrt(vvpsi21))
print("stanard deviation of psi31:", np.sqrt(vvpsi31))
print("stanard deviation of psi41:", np.sqrt(vvpsi41))
print("stanard deviation of psi51:", np.sqrt(vvpsi51))
print("stanard deviation of psi12:", np.sqrt(vvpsi12))
print("stanard deviation of psi22:", np.sqrt(vvpsi22))
print("stanard deviation of psi32:", np.sqrt(vvpsi32))
print("stanard deviation of psi42:", np.sqrt(vvpsi42))
print("stanard deviation of psi52:", np.sqrt(vvpsi52))
print("mean of phi1:", mmphi1)
print("mean of phi2:", mmphi2)
print("stanard deviation of phi1:", np.sqrt(vvphi1))
print("stanard deviation of phi2:", np.sqrt(vvphi2))
print("mean of mu1:", mmmu1)
print("mean of mu2:", mmmu2)
print("stanard deviation of mu1:", np.sqrt(vvmu1))
print("stanard deviation of mu2:", np.sqrt(vvmu2))
print("stanard deviation of sigma1:", sqrt_var_bsig1)
print("stanard deviation of sigma2:", sqrt_var_bsig2)
print("stanard deviation of sigma3:", sqrt_var_bsig3)
print("stanard deviation of sigma4:", sqrt_var_bsig4)
print("stanard deviation of sigma5:", sqrt_var_bsig5)
print("mean of sigma1:", mbsig1)
print("mean of sigma2:", mbsig2)
print("mean of sigma3:", mbsig3)
print("mean of sigma4:", mbsig4)
print("mean of sigma5:", mbsig5)

mean of lamda1: 9.95760865101711e-07
mean of lamda2: -8.281413457797196e-05
mean of lamda3: -0.014467960667868247
mean of lamda4: -0.00013135563424465947
mean of lamda5: -0.0007384407776915394
stanard deviation of lamda1: 1.0437705927640504e-05
stanard deviation of lamda2: 1.2995606340015074e-05
stanard deviation of lamda3: 2.321061643080639e-05
stanard deviation of lamda4: 1.2393825438831702e-05
stanard deviation of lamda5: 1.1426824991890204e-05
mean of psi11: 0.004218921559789076
mean of psi21: 0.03099046836537819
mean of psi31: 0.0032891786735667755
mean of psi41: -0.0037651459113793554
mean of psi51: 0.012976799020934503
mean of psi12: 0.0018625936686717263
mean of psi22: 0.004746312618659771
mean of psi32: -0.0010252614836818285
mean of psi42: -0.0018401089465949502
mean of psi52: 0.004229692017536591
stanard deviation of psi11: 0.5530388711413227
stanard deviation of psi21: 0.04585155475890119
stanard deviation of psi31: 0.004972164884994585
stanard deviation of psi41: 0.0195636